In [757]:
### IMPORT DATA

from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import string
import csv
import sys

p = requests.get('http://www.presidentsusa.net/presvplist.html').text
p = BeautifulSoup(p)
p2 = requests.get('http://www.presidentsusa.net/partyofpresidents.html').text
p2 = BeautifulSoup(p2)

In [246]:
### print p.prettify()[4400:5000]

In [758]:
### GETS LIST AND SEPARATES PRESIDENTS FROM VPs

p_td = p.find_all('td')
pres = []
vp = []
for tag in p_td:
    try:
        int(tag.get_text()[0])
        pres.append(tag.get_text())
    except (ValueError):
        vp.append(tag.get_text())

In [759]:
### RETRIEVES LIST OF P AND VP BY PARTY

party = p2.find_all('div',{'class':"col-xs-3"})
#party

In [760]:
### RETURNS LIST OF PRESIDENTS AND VPs BY PARTY

for i in range(len(party)):
    if i == 0:
        rep = party[i].get_text().split('\n')
    elif i == 1:
        dem = party[i].get_text().split('\n')
    elif i == 2:
        other = party[i].get_text().split('\n')
    else:
        etc = party[i].get_text().split('\n')

In [761]:
### CONVERT DATA FROM UNICODE TO STRINGS

pres1 = map(lambda x: str(x),pres)
# pres1
vp1 = map(lambda x: str(x),vp)
# vp1
rep1 = map(lambda x: str(x),rep)
# rep1
dem1 = map(lambda x: str(x),dem)
# dem1
other1 = map(lambda x: str(x),other)
# other1

In [766]:
### USE REGEX TO CLEAN AND PARSE PRESIDENT'S DATA

# Presidents data
pres2=[]
for pre in pres1:
    temp0 = re.sub(' \(','(',pre)
    temp1 = re.sub('[1-90]+\. ',"",temp0) # TAKE OUT CHARACTERS BEFORE THE NAME
    temp2 = re.sub('[(\(-]',",",temp1) # TAKE OUT SPEC CHARACTERS REPLACE WITH ,
    temp3 = re.sub('\)',"",temp2)     # TAKE OUT LAST PARENTHESIS
    pres2 = pres2 + [temp3]
pres2[0:5]

['George Washington,1789,1797',
 'John Adams,1797,1801',
 'Thomas Jefferson,1801,1809',
 'James Madison,1809,1817',
 'James Monroe,1817,1825']

In [807]:
### SPLIT PRESIDENT RECORDS INTO LIST WITH NAME, YEAR STARTED, AND YEAR ENDED

pres3 = [x.split(',') for x in pres2]

### FIX PRESIDENT RECORDS WITH NO END YEAR

for i in range(len(pres3)):
    if len(pres3[i])==2:
        pres3[i] = pres3[i]+[pres3[i][1]]

for i in range(len(pres3)):
    try:
        pres3[i][1:3] = map(int,pres3[i][1:3])
    except ValueError:
        pres3[i][1:3] = [int(pres3[i][1])] + [2016]
        
pres3[0:5]

[['George Washington', 1789, 1797],
 ['John Adams', 1797, 1801],
 ['Thomas Jefferson', 1801, 1809],
 ['James Madison', 1809, 1817],
 ['James Monroe', 1817, 1825]]

In [808]:
### FIX MULTIPLE ENTRIES IN SAME LINE FOR VP DATA
###     AND DELETE ROWS THAT CONTAIN ADS AND OTHER
###     GARBAGE DATA

vp2 = [x.split('\n') for x in vp1]
vp3 =[]
for vp in vp2:
    for i in range(len(vp)):
        if vp[i]:
            if vp[i][0] not in [string.punctuation,'\r','(']:
                vp3 = vp3 + [vp[i]]
    
vp3[0:5]

['John Adams (1789-1797)',
 'Thomas Jefferson (1797-1801)',
 'Aaron Burr (1801-1805)',
 'George Clinton (1805-1809)',
 'George Clinton (1809-1812)']

In [809]:
### CLEAN UP VICE PRESIDENT DATA USING REGEX

vp4=[]
for vp in vp3:
    temp0 = re.sub(' \(','(',vp)
    temp1 = re.sub('[\(-]',",",temp0) # TAKE OUT SPEC CHARACTERS REPLACE WITH ,
    temp2 = re.sub('\)',"",temp1)     # TAKE OUT LAST PARENTHESIS
    vp4 = vp4 + [temp2]
vp4[0:5]

['John Adams,1789,1797',
 'Thomas Jefferson,1797,1801',
 'Aaron Burr,1801,1805',
 'George Clinton,1805,1809',
 'George Clinton,1809,1812']

In [810]:
### SPLIT VICE PRESIDENT RECORDS INTO LIST 
###     WITH NAME, YEAR STARTED, AND YEAR ENDED

vp5 = [x.split(',') for x in vp4]    

### FIX VP RECORDS WITH NO END YEAR

for i in range(len(vp5)):
    if len(vp5[i])==2:
        vp5[i] = vp5[i]+[vp5[i][1]]

for i in range(len(vp5)):
    try:
        vp5[i][1:3] = map(int,vp5[i][1:3])
    except ValueError:
        vp5[i][1:3] = [int(vp5[i][1])] + [2016]

vp5[0:5]

[['John Adams', 1789, 1797],
 ['Thomas Jefferson', 1797, 1801],
 ['Aaron Burr', 1801, 1805],
 ['George Clinton', 1805, 1809],
 ['George Clinton', 1809, 1812]]

In [811]:
### CLEAN UP LIST BY PARTY

parties = ['Republican','Democrat','Federalist','Democratic-Republican','Whig','Union']
rep2 = []
for x in rep1:
    if x and (x not in parties):
        rep2 = rep2 + [x]

dem2 = []
for x in dem1:
    if x and (x not in parties):
        dem2 = dem2 + [x]

other2 = []
for x in other1:
    if x and (x not in parties):
        other2 = other2 + [x]


In [812]:
### MERGE 3 LISTS OF PARTIES INTO ONE
rep3 = map(lambda x: [x]+['Republican'],rep2)
dem3 = map(lambda x: [x]+['Democrat'],dem2)
other3 = map(lambda x: [x]+['OtherParties'],other2)
partylist=rep3+dem3+other3

In [813]:
### CHECK IF NAMES IN PRESIDENT LIST MATCH THOSE IN PARTY LIST
### FIVE NAMES DONT MATCH WHICH HAVE TO BE MANUALLY CORRECTED

all2 = []
for p in partylist:
    all2 = all2 + [p[0]]

for x in range(len(pres3)):
    if pres3[x][0] not in all2:
        print x,pres3[x]

19 ['James A. Garfield', 1881, 1881]
28 ['Warren G. Harding', 1921, 1923]
32 ['Harry S. Truman', 1945, 1953]
33 ['Dwight D. Eisenhower', 1953, 1961]
35 ['Lyndon B. Johnson', 1963, 1969]


In [814]:
# CORRECT NAMES MISMATCH
partylist[3][0] = pres3[19][0]
partylist[9][0] = pres3[28][0]
partylist[26][0] = pres3[32][0]
partylist[12][0] = pres3[33][0]
partylist[28][0] = pres3[35][0]

# TEST AGAIN FOR MISMATCH
all2 = []
for p in partylist:
    all2 = all2 + [p[0]]

for x in range(len(pres3)):
    if pres3[x][0] not in all2:
        print x,pres3[x]

In [815]:
# Check If Names Match In President and VP List
# All exceptions didnt become VP so there is a name match
all3 = []
for p in vp5:
    all3 = all3 + [p[0]]

for x in range(len(pres3)):
    if pres3[x][0] not in all3:
        print x,pres3[x]

0 ['George Washington', 1789, 1797]
3 ['James Madison', 1809, 1817]
4 ['James Monroe', 1817, 1825]
5 ['John Quincy Adams', 1825, 1829]
6 ['Andrew Jackson', 1829, 1837]
8 ['William Henry Harrison', 1841, 1841]
10 ['James K. Polk', 1845, 1849]
11 ['Zachary Taylor', 1849, 1850]
13 ['Franklin Pierce', 1853, 1857]
14 ['James Buchanan', 1857, 1861]
15 ['Abraham Lincoln', 1861, 1865]
17 ['Ulysses S. Grant', 1869, 1877]
18 ['Rutherford B. Hayes', 1877, 1881]
19 ['James A. Garfield', 1881, 1881]
21 ['Grover Cleveland', 1885, 1889]
22 ['Benjamin Harrison', 1889, 1893]
23 ['Grover Cleveland', 1893, 1897]
24 ['William McKinley', 1897, 1901]
26 ['William Howard Taft', 1909, 1913]
27 ['Woodrow Wilson', 1913, 1921]
28 ['Warren G. Harding', 1921, 1923]
30 ['Herbert Hoover', 1929, 1933]
31 ['Franklin D. Roosevelt', 1933, 1945]
33 ['Dwight D. Eisenhower', 1953, 1961]
34 ['John F. Kennedy', 1961, 1963]
38 ['Jimmy Carter', 1977, 1981]
39 ['Ronald Reagan', 1981, 1989]
41 ['Bill Clinton', 1993, 2001]
42 ['G

In [816]:
### Prepare data for export

masterp = pres3[ : ]   # Initialize to president's list
for i in range(len(testp)):
    masterp[i].insert(1,1)         # Insert P Indicator
    masterp[i].insert(2,0)         # Insert VP Indicator
    masterp[i].insert(3,testp[i][4]-testp[i][3])   # Years as President
    for j in range(len(partylist)):
        if masterp[i][0] == partylist[j][0]:
            masterp[i].insert(1,partylist[j][1])
    
pres3[0:5]

[['George Washington', 'OtherParties', 1, 0, 8, 1789, 1797],
 ['John Adams', 'OtherParties', 1, 0, 4, 1797, 1801],
 ['Thomas Jefferson', 'OtherParties', 1, 0, 8, 1801, 1809],
 ['James Madison', 'OtherParties', 1, 0, 8, 1809, 1817],
 ['James Monroe', 'OtherParties', 1, 0, 8, 1817, 1825]]

In [817]:
### EXPORT DATA

f1 = "../Data_Science/test/president.csv"
with open(f1, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(masterp)

In [820]:
# pres3